# 🚀 HVAC AI - Remote Inference Server (v2 - Configured)

This notebook acts as a GPU-powered backend server for the `hvac-ai` platform. It clones the repository, installs dependencies, and uses `ngrok` to create a public URL for your local frontend. This version is enhanced to use a `.env` file for secure and portable configuration.

### Phase 1: Environment Setup

In [ ]:
# 1. Clone the GitHub repository
!git clone https://github.com/elliotttmiller/hvac-ai.git

# 2. Change the current working directory to the cloned repo
%cd hvac-ai

In [ ]:
# 3. Install all required Python packages

# Web server, tunneling, and configuration dependencies
!pip install flask flask-cors pyngrok python-dotenv --quiet

# Machine learning and SAM dependencies
!pip install torch torchvision --quiet
!pip install opencv-python pycocotools matplotlib onnxruntime onnx --quiet
!pip install git+https://github.com/facebookresearch/segment-anything.git --quiet

print("✅ All dependencies installed successfully.")

### Phase 2: Create and Configure `.env` File

This is the most important step for configuring your server securely.

1.  **Get your `ngrok` authtoken** from your dashboard: https://dashboard.ngrok.com/get-started/your-authtoken
2.  **Run the cell below**, pasting your `ngrok` token and the correct Google Drive path to your fine-tuned model (`.pth` file) where indicated.

In [ ]:
%%writefile .env
# .env file for HVAC AI Inference Server

# 1. Paste your secret token from the ngrok dashboard
NGROK_AUTHTOKEN="YOUR_NGROK_AUTHTOKEN_HERE"

# 2. Update with the full path to your fine-tuned model in Google Drive
MODEL_PATH="/content/drive/MyDrive/sam_finetuning_results/best_model_expert_v1.pth"


### Phase 3: Authenticate `ngrok`

In [ ]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file we just created
load_dotenv()

# Get the token
ngrok_token = os.getenv("NGROK_AUTHTOKEN")

if ngrok_token and "YOUR_NGROK_AUTHTOKEN_HERE" not in ngrok_token:
    print("NGROK_AUTHTOKEN loaded successfully. Authenticating...")
    !ngrok authtoken {ngrok_token}
else:
    print("❌ ERROR: NGROK_AUTHTOKEN not found or not updated in the .env file.")
    print("👉 Please edit the cell in Phase 2 with your real token and run it again.")

### Phase 4: Prepare and Launch the Inference Server

#### ⚠️ **Action Required: Update the Backend Script**

The final step is to ensure your `backend/app.py` script is configured to load variables from the `.env` file.

1. In the file browser on the left, navigate to `hvac-ai` -> `backend`.
2. Double-click on `app.py` to open it.
3. **Replace the entire content** of `app.py` with the provided professional, `.env`-aware code from our discussion. This code should import `dotenv` and use `os.getenv("MODEL_PATH")`.

In [ ]:
from google.colab import drive
from pyngrok import ngrok
import os

# --- 1. Mount Google Drive to access the model file ---
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("✅ Drive mounted.")

# --- 2. Set up the ngrok tunnel ---
PORT = 5000
ngrok.kill()
public_url = ngrok.connect(PORT).public_url
print("-"*50)
print(f"✅ Your inference server is live at: {public_url}")
print("👉 Use this URL as the API endpoint in your local frontend application.")
print("-"*50)

# --- 3. Run the Flask Application ---
# The script will now read its configuration from the .env file.
# This cell will run continuously and display live logs from your server.
print("\n🚀 Starting the backend server... (Logs will appear below)")
!python backend/app.py
